In [3]:
from netCDF4 import Dataset
import numpy as np
import xarray as xr

# Load the dataset
file_path = "D:/SIH 2024/S5P_OFFL_L2__NO2____20241030T213820_20241030T231949_36526_03_020701_20241101T132516.nc"
nc_data = Dataset(file_path, mode='r')

# Access the PRODUCT group and extract necessary variables
product_group = nc_data.groups['PRODUCT']
no2_data = product_group.variables['nitrogendioxide_tropospheric_column'][:]
latitude = product_group.variables['latitude'][:]
longitude = product_group.variables['longitude'][:]

# Convert to xarray for easier handling
data = xr.open_dataset(file_path, group='PRODUCT')


In [4]:
import numpy as np
import matplotlib.pyplot as plt

def box_count(img, box_sizes):
    counts = []
    for size in box_sizes:
        count = 0
        for i in range(0, img.shape[0], size):
            for j in range(0, img.shape[1], size):
                if np.sum(img[i:i+size, j:j+size]) > 0:
                    count += 1
        counts.append(count)
    return np.array(counts)

# Example of downscaling data (assuming `no2_data` is a 2D grid)
box_sizes = np.logspace(1, np.log10(min(no2_data.shape)), num=20, dtype=int)
counts = box_count(no2_data, box_sizes)

# Fit the fractal dimension to the box-counting data
log_box_sizes = np.log(box_sizes)
log_counts = np.log(counts)
fractal_dim = -np.polyfit(log_box_sizes, log_counts, 1)[0]

print("Fractal Dimension:", fractal_dim)


Fractal Dimension: 0.9949752054732586


In [6]:
print(len(latitude), len(longitude))

print("Latitude:", latitude)
print("Longitude:", longitude)
print("Latitude shape:", latitude.shape)
print("Longitude shape:", longitude.shape)



1 1
Latitude: [[[-64.44701  -64.46644  -64.48534  ... -56.49504  -56.433323 -56.370667]
  [-64.49422  -64.513695 -64.53262  ... -56.533787 -56.472015 -56.409306]
  [-64.5414   -64.56091  -64.57989  ... -56.57254  -56.510715 -56.447952]
  ...
  [ 68.987206  69.071396  69.15442  ...  86.64859   86.573524  86.497154]
  [ 68.98504   69.0692    69.15218  ...  86.63653   86.5619    86.48596 ]
  [ 68.98283   69.06694   69.14989  ...  86.623764  86.54958   86.47408 ]]]
Longitude: [[[ 63.460926  63.26171   63.065014 ...  14.401759  14.298953  14.195079]
  [ 63.46975   63.270203  63.073177 ...  14.344166  14.241354  14.137475]
  [ 63.478897  63.279015  63.08166  ...  14.286627  14.183811  14.079929]
  ...
  [125.538086 125.494125 125.45045  ... -37.894524 -38.424107 -38.9377  ]
  [125.39776  125.35324  125.30902  ... -37.11055  -37.65573  -38.184658]
  [125.25748  125.21242  125.16765  ... -36.332294 -36.892628 -37.43648 ]]]
Latitude shape: (1, 4173, 450)
Longitude shape: (1, 4173, 450)


In [7]:
latitude_flat = latitude.flatten()
longitude_flat = longitude.flatten()
print("Flattened Latitude shape:", latitude_flat.shape)
print("Flattened Longitude shape:", longitude_flat.shape)


Flattened Latitude shape: (1877850,)
Flattened Longitude shape: (1877850,)


In [8]:
from scipy.interpolate import griddata

# Create a grid for new latitude and longitude
new_latitude = np.linspace(min(latitude_flat), max(latitude_flat), 50)  # Use smaller grid size for testing
new_longitude = np.linspace(min(longitude_flat), max(longitude_flat), 50)

# Create meshgrid for interpolation
grid_lat, grid_lon = np.meshgrid(new_latitude, new_longitude)

# Interpolate NO2 data
downscaled_no2 = griddata(
    (latitude_flat, longitude_flat),
    no2_data.flatten(),  # Flatten NO2 data
    (grid_lat, grid_lon),
    method='cubic'
)
print("Downscaled NO2 Data:", downscaled_no2)


Downscaled NO2 Data: [[           nan            nan            nan ...            nan
             nan            nan]
 [           nan 6.62399899e+30 1.42124670e+38 ... 9.96920997e+36
  9.96920997e+36            nan]
 [           nan 2.16372902e+26 5.49077581e+37 ... 9.96920997e+36
  9.96920997e+36            nan]
 ...
 [           nan 2.36034094e+35 3.45954588e+38 ... 9.96920997e+36
  9.96920997e+36            nan]
 [           nan 1.05716058e+33 2.24115793e+38 ... 9.96920997e+36
  9.96920997e+36            nan]
 [           nan            nan            nan ...            nan
             nan            nan]]
